from "The Neo4j GenAI Package for Python - Getting started with retrieval and GraphRAG" by Will Tai

https://medium.com/neo4j/the-neo4j-genai-package-for-python-bbb2bd2ad3b3

<a target="_blank" href="https://colab.research.google.com/github/v2belleville/notebooks/blob/main/Neo4j%20GenAI%20Package%20for%20Python.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

In [ ]:
from neo4j import GraphDatabase

In [ ]:
# Demo database credentials
URI = "neo4j+s://demo.neo4jlabs.com"
AUTH = ("recommendations", "recommendations")

In [2]:
# Connect to Neo4j database
driver = GraphDatabase.driver(URI, auth=AUTH)
session = driver.session()

In [3]:
import os
os.environ["OPENAI_API_KEY"]="sk-..."

In [5]:
session.run("show indexes YIELD * WHERE type='VECTOR' ").values()

[[11,
  'moviePlotsEmbedding',
  'ONLINE',
  100.0,
  'VECTOR',
  'NODE',
  ['Movie'],
  ['plotEmbedding'],
  'vector-1.0',
  None,
  neo4j.time.DateTime(2024, 7, 30, 16, 5, 12, 677000000, tzinfo=<UTC>),
  73,
  neo4j.time.DateTime(2024, 1, 9, 14, 16, 32, 599000000, tzinfo=<UTC>),
  {'indexProvider': 'vector-1.0',
   'indexConfig': {'vector.dimensions': 1536,
    'vector.similarity_function': 'cosine'}},
  '',
  "CREATE VECTOR INDEX `moviePlotsEmbedding` FOR (n:`Movie`) ON (n.`plotEmbedding`) OPTIONS {indexConfig: {`vector.dimensions`: 1536,`vector.similarity_function`: 'cosine'}, indexProvider: 'vector-1.0'}"],
 [13,
  'moviePostersEmbedding',
  'ONLINE',
  100.0,
  'VECTOR',
  'NODE',
  ['Movie'],
  ['posterEmbedding'],
  'vector-1.0',
  None,
  None,
  0,
  neo4j.time.DateTime(2024, 1, 9, 14, 16, 51, 774000000, tzinfo=<UTC>),
  {'indexProvider': 'vector-1.0',
   'indexConfig': {'vector.dimensions': 512,
    'vector.similarity_function': 'cosine'}},
  '',
  "CREATE VECTOR INDEX `movi

In [ ]:
from neo4j_genai.retrievers import VectorRetriever
from neo4j_genai.embeddings.openai import OpenAIEmbeddings

In [ ]:
embedder = OpenAIEmbeddings(model="text-embedding-ada-002")
retriever = VectorRetriever(
    driver,
    index_name="moviePlotsEmbedding",
    embedder=embedder,
    return_properties=["title", "plot"],
)

In [7]:
query_text = "A movie about the famous sinking of the Titanic"
retriever_result = retriever.search(query_text=query_text, top_k=3)
print(retriever_result)

items=[RetrieverResultItem(content="{'title': 'Titanic', 'plot': 'An unhappy married couple deal with their problems on board the ill-fated ship.'}", metadata={'score': 0.9450652599334717, 'nodeLabels': None, 'id': None}), RetrieverResultItem(content="{'title': 'Night to Remember, A', 'plot': 'An account of the ill-fated maiden voyage of RMS Titanic in 1912.'}", metadata={'score': 0.9428615570068359, 'nodeLabels': None, 'id': None}), RetrieverResultItem(content="{'title': 'Titanic', 'plot': 'A seventeen-year-old aristocrat falls in love with a kind, but poor artist aboard the luxurious, ill-fated R.M.S. Titanic.'}", metadata={'score': 0.9422949552536011, 'nodeLabels': None, 'id': None})] metadata={'__retriever': 'VectorRetriever'}


## simple GraphRAG pipeline

In [ ]:
from neo4j_genai.llm import OpenAILLM
from neo4j_genai.generation import GraphRAG

# LLM
# Note: the OPENAI_API_KEY must be in the env vars
genLLM = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

# Initialize the RAG pipeline
rag = GraphRAG(retriever=retriever, llm=genLLM)

In [19]:
# Query the graph
query_text = "What movies are sad romances?"
response = rag.search(query=query_text, retriever_config={"top_k": 5})

response

RagResultModel(answer='"Autumn in New York" and "Bed of Roses" are sad romances. "Autumn in New York" is about an aging playboy who falls for a sweet, but terminally ill, young woman, while "Bed of Roses" involves a young career girl who is swept off her feet by a shy florist.', retriever_result=None)

In [18]:
# Query the graph
query_text = "What movies are about politics?"
response = rag.search(query=query_text, retriever_config={"top_k": 5})

response

RagResultModel(answer="The movies about politics are:\n\n1. **Mr. Smith Goes to Washington** - A naive man is appointed to fill a vacancy in the US Senate and faces political corruption.\n2. **American President, The** - A comedy-drama about a widowed U.S. president and a lobbyist who fall in love amidst political challenges.\n3. **All the King's Men** - The rise and fall of a corrupt politician who uses populist appeal to retain power.\n4. **Perfect Candidate, A** - A documentary following Oliver North's unsuccessful 1994 bid for a Virginia Senate seat, highlighting the campaign's inner workings.\n5. **Wag the Dog** - A spin-doctor and a Hollywood producer fabricate a war to cover up a presidential sex scandal shortly before an election.", retriever_result=None)

In [17]:
# Query the graph
query_text = "What movies are about politics but not corruption ?"
response = rag.search(query=query_text, retriever_config={"top_k": 5})
response

RagResultModel(answer='"American President, The" is a movie about politics but not corruption. It is a comedy-drama about a widowed U.S. president and a lobbyist who fall in love, dealing with the perception of their relationship in the political arena.', retriever_result=None)